# 第2章-自然语言和单词的分布式表示

## 2.1 什么是自然语言处理

自然语言（natural language）

自然语言处理（Natural Language Processing，NLP），顾名思义，就是处理自然语言的科学。
它是一种能够让计算机理解人类语言的技术。

自然语言处理的目标就是让计算机理解人说的话，进而完成对我们有帮助的事情。

### 单词含义

我们的语言是由文字构成的，而语言的含义是由单词构成的。换句话说，单词是含义的最小单位。

单词含义的表示方法：
* 基于同义词词典的方法
* 基于计数的方法
* 基于推理的方法（word2vec）

[CS224d: Deep Learning for Natural Language Processing](http://cs224d.stanford.edu/)

## 2.2 同义词字典

![](../images/图2-1.同义词的例子：car、auto和automobile等都是表示“汽车”的同义词.PNG)
图2-1.同义词的例子：car、auto和automobile等都是表示“汽车”的同义词

![](../images/图2-2.根据各单词的含义，基于上位-下位关系形成的图.PNG)
图2-2.根据各单词的含义，基于上位-下位关系形成的图

### 2.2.1 WordNet

在自然语言处理领域，最著名的同义词词典是 WordNet。
WordNet 是普林斯顿大学于 1985 年开始开发的同义词词典，迄今已用于许多研究，
并活跃于各种自然语言处理应用中。

### 2.2.2 同义词词典的问题

#### 难以顺应时代的变化
#### 人力成本
#### 无法表示单词的微妙差异

## 2.3 基于计数的方法

**语料库**（corpus）就是大量的文本数据。
语料库并不是胡乱收集数据，一般收集的都是用于自然语言处理研究和应用的文本数据。

### 2.3.1 基于Python的语料库预处理

In [3]:
text = 'You say goodbye and I say hello.'
text.lower()
text = text.replace('.', ' .')
text

'You say goodbye and I say hello .'

In [4]:
words = text.split(' ')
words

['You', 'say', 'goodbye', 'and', 'I', 'say', 'hello', '.']

In [5]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [6]:
id_to_word

{0: 'You', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'I', 5: 'hello', 6: '.'}

In [7]:
word_to_id

{'You': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'I': 4, 'hello': 5, '.': 6}

In [8]:
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [9]:
from common.util import *

text = 'You say goodbye and I sat hello.'
# corpus（语料库） 单词ID列表，word_to_id 单词到单词ID的字典，id_to_word 单词ID到单词的字典
corpus, word_to_id, id_to_word = preprocess(text)
corpus

array([0, 1, 2, 3, 4, 5, 6, 7])

上述一系列处理实现为 preprocess() 函数（[common/util.py](../common/util.py)）。

### 2.3.2 单词的分布式表示

颜色也可以通过 RGB（Red/Green/Blue）三原色分别存在多少来表示。前者为不同的颜色赋予不同的名字，有
多少种颜色，就需要有多少个不同的名字；后者则将颜色表示为三维向量。

在单词领域构建紧凑合理的向量表示。
能准确把握单词含义的向量表示。在自然语言处理领域，称为**分布式表示**。

### 2.3.3 分布式假设

**分布式假设**（distributional hypothesis）：某个单词的含义由它周围的单词组成。

分布式假设所表达的理念非常简单。单词本身没有含义，单词含义由它所在的上下文（语境）形成。

上下文指某个单词（关注词）周围的单词。

![](../images/图2-3.窗口大小为2的上下文例子.PNG)
图2-3.窗口大小为2的上下文例子。在关注goodbye时，将其左右各2个单词用作上下文

将上下文的大小（即周围的单词有多少个）称为**窗口大小**（window size）。

### 2.3.4 共现矩阵

基于统计\计数的方法：在关注某个单词的情况下，对它的周围出现多少次什么单词进行计数，然后再汇总。


In [1]:
import sys
import numpy as np
from common.util import preprocess

sys.path.append('..')

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


![](../images/图2-4.单词you的上下文.PNG)
图2-4.单词you的上下文

![](../images/图2-5.用表格表示单词you的上下文中包含的单词的频数.PNG)
图2-5.用表格表示单词you的上下文中包含的单词的频数

![](../images/图2-6.用表格表示单词say的上下文中包含的单词的频数.PNG)
图2-6.用表格表示单词say的上下文中包含的单词的频数

![](../images/图2-7.用表格汇总各个单词的上下文中包含的单词的频数.PNG)
图2-7.用表格汇总各个单词的上下文中包含的单词的频数

图 2-7 的表格呈矩阵状，所以称为**共现矩阵**（co-occurence matrix）。

In [5]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0],
], dtype=np.int32)
print(C[0]) # 单词ID为0的向量
print(C[4])
print(C[word_to_id['goodbye']]) # goodbye的向量

[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]
[0 1 0 1 0 0 0]


直接从语料库生成共现矩阵的函数 create_co_matrix(corpus, vocab_size, window_size=1)，
其中参数 corpus 是单词 ID 列表，参数 vocab_size 是词汇个数，window_size 是窗口大小（[common/util.py](../common/util.py)）。

### 2.3.5 向量间的相似度

测量向量间的相似度有很多方法，其中具有代表性的方法有向量内积或欧式距离等。

**余弦相似度**（cosine similarity）
$$
    \boldsymbol{x} = (x_1, x_2, x_3, \cdots , x_n)
    \\
    \boldsymbol{y} = (x_1, x_2, x_3, \cdots , x_n)
    \\
    similarity(\boldsymbol{x}, \boldsymbol{y}) = \frac{\boldsymbol{x} \boldsymbol{\cdot} \boldsymbol{y}}{ \left\| \boldsymbol{x} \right\| \left\| \boldsymbol{y} \right\|} =
    \frac{x_1y_1 + \cdots + x_ny_n}{\sqrt{x_1^2 + \cdots + x_n^2}\sqrt{y_1^2 + \cdots + y_n^2}}
    \tag{2.1}
$$

余弦相似度直观地表示了“两个向量在多大程度上指向同一方向”。
两个向量完全指向相同的方向时，余弦相似度为 1；完全指向相反
的方向时，余弦相似度为 −1。

In [ ]:
# 余弦相似度，加eps防止除数为0
def cos_similarity(x, y, eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps) # x的正规化
    ny = y / np.sqrt(np.sum(y**2) + eps) # y的正规化
    return np.dot(nx, ny)

### 2.3.6 相似单词的排序

most_similar() 函数（ common/util.py）。

[ch02/most_similar.py](../ch02/most_similar.py)

most_similar(query, word_to_id, id_to_word, word_matrix, top=5)

![](../images/表2-1.most_similar()函数的参数.PNG)
表2-1.most_similar()函数的参数

## 2.4 基于计数的方法的改进

### 2.4.1 点互信息（Pointwise Mutual Information，PMI）

对于随机变量 $x$ 和 $y$，它们的PMI定义：
$$
    PMI(x,y) = \log_2 \frac{P(x,y)}{P(x)P(y)} \tag{2.2}
$$
$P(x)$ 表示 $x$ 发生的概率，
$P(y)$ 表示 $y$ 发生的概率，
$P(x,y)$ 表示 $x$ 和 $y$ 同时发生的概率。
PMI的值越高，相关性越强。


使用共现矩阵（其元素表示单词共现的次数）来重写式(2.2)。
这里，将共现矩阵表示为 $\mathbf{C}$，将单词 $x$ 和 $y$ 的共现次数表示为 $\mathbf{C}(x, y)$，将
单词 $x$ 和 $y$ 的出现次数分别表示为 $\mathbf{C}(x)$、$\mathbf{C}(y)$，将语料库的单词数量记为
$N$，则式 (2.2) 可以重写为：
$$
    PMI(x,y) = \log_2 \frac{P(x,y)}{P(x)P(y)}
    = \log_2 \frac{\frac{C(x,y)}{N}}{\frac{C(x)}{N} \frac{C(y)}{N}}
    = \log_2 \frac{C(x,y) \cdot N}{C(x)C(y)}
    \tag{2.3}
$$


$$
    PMI("the","car") = \log_2 \frac{10 \cdot 10000}{1000 \cdot 20} \approx 2.32 \tag{2.4}
    \\
    PMI("car","drive") = \log_2 \frac{5 \cdot 10000}{20 \cdot 10} \approx 7.97 \tag{2.5}
$$


当两个单词共现次数是 $0$ 时，$log_20 = -\infty$。
实践中会使用**正点互信息**（Postive PMI，PPMI）。

$$
    PPMI(x,y) = \max(0, PMI(x,y)) \tag{2.6}
$$

实现将共现矩阵转化为PPMI 矩阵的函数。我们把这个函数称为 ppmi(C, verbose=False, eps=1e-8)
（ [common/util.py](../common/util.py)）。

将共现矩阵转化为 PPMI 矩阵，可以像下面这样进行实现（[ch02/ppmi.py](../ch02/ppmi.py)）。

这个 PPMI 矩阵还是存在一个很大的问题，那就是随着语料库的词汇量增加，各个单词向量的维数也会增加。

### 2.4.2 降维

**降维**（dimensionality reduction），在尽量保留“重要信息”的基础上减少向量维度。

![](../images/图2-8.降维示意图.PNG)
图2-8.降维示意图：发现重要的轴（数据分布广的轴），将二维数据表示为一维数据

向量中的大多数元素为 0 的矩阵（或向量）称为稀疏矩阵（或稀疏向量）。
从稀疏向量中找出重要的轴，用更少的维度对其进行重新表示。结果，
稀疏矩阵就会被转化为大多数元素均不为0的密集矩阵。这个密集矩阵就是我们想要的单词的分布式表示。

降维的方法有很多，这里我们使用奇异值分解（Singular Value Decomposition，SVD）

**奇异值分解**（Singular Value Decomposition, SVD）

SVD将任意矩阵分解为3个矩阵的乘积：
$$
    X = USV^T \tag{2.7}
$$

SVD 将任意的矩阵 $X$ 分解为 $U$、$S$、$V$ 这 3 个矩阵的乘积，
其中 $U$ 和 $V$ 是列向量彼此正交的正交矩阵，
$S$ 是除了对角线元素以外其余元素均为 $0$ 的对角矩阵。

![](../images/图2-9.基于SVD的矩阵变换（白色部分表示元素为0）.PNG)
图2-9.基于SVD的矩阵变换（白色部分表示元素为0）

![](../images/图2-10.基于SVD的降维示意图.PNG)
图2-10.基于SVD的降维示意图

### 2.4.3 基于SVD的降维

创建一个共现矩阵，将其转化为 PPMI 矩阵，然后对其进行 SVD（ [ch02/count_method_small.py](../ch02/count_method_small.py)）

![](../images/图2-11.对共现矩阵执行SVD，并在图上绘制各个单词的二维向量（i和goodbye重叠）.PNG)
图2-11　对共现矩阵执行SVD，并在图上绘制各个单词的二维向量（i和goodbye重叠）

* 根据操作系统的种类或Matplotlib版本的不同，输出的图可能和图2-11所有不同。

### 2.4.4 PTB数据集

Penn Treebank 语料库

[ch02/show_ptb.py](../ch02/show_ptb.py)

![](../images/图2-12.PTB语料库（文本文件）的例子.PNG)
图2-12 PTB语料库（文本文件）的例子

### 2.4.5 基于PTB数据集的平评价

[ch02/count_method_big.py](../ch02/count_method_big.py)

## 小结

## 本章所学的内容

* 使用 WordNet 等同义词词典，可以获取近义词或测量单词间的相似度等
* 使用同义词词典的方法存在创建词库需要大量人力、新词难更新等问题
* 目前，使用语料库对单词进行向量化是主流方法
* 近年来的单词向量化方法大多基于“单词含义由其周围的单词构成”这一分布式假设
* 在基于计数的方法中，对语料库中的每个单词周围的单词的出现频数进行计数并汇总（= 共现矩阵）
* 通过将共现矩阵转化为 PPMI 矩阵并降维，可以将大的稀疏向量转变为小的密集向量
* 在单词的向量空间中，含义上接近的单词距离上理应也更近